# コンピュータービジョンサービスによる画像解析

![A robot holding a picture](./images/computer_vision.jpg)

**コンピュータービジョン**とは、人工知能（AI）の一分野であり、カメラを使ってリアルタイムで世界を見たり、画像やビデオを分析して世界を「見る」ことができるAIシステムの開発を目的としています。コンピュータビジョンとは、カメラで撮影した画像をリアルタイムに解析したり、画像や動画を解析したりすることで、世界を「見る」ことができるAIシステムを開発する学問です。

## Computer Vision Cognitive Services を使用する

Microsoft Azureには、一般的なAI機能をカプセル化した**Cognitive Services**が数多く含まれており、コンピュータビジョンソリューションの構築に役立つものもあります。

**Computer Vision** Cognitive Serviceは、Azureでコンピュータビジョンを探求するための明白な出発点となります。このサービスは、事前に訓練された機械学習モデルを使用して画像を分析し、画像に関する情報を抽出します。

例えば、Northwind Traders社が「スマートストア」の導入を決定したとします。AIサービスが店内を監視し、サポートを必要とする顧客を特定し、従業員に支援を指示するというものです。コンピュータビジョンサービスを利用することで、店舗内のカメラで撮影した画像を解析し、何を描いているのか意味のある説明を提供することができます。

### Cognitive Services リソースの作成

はじめに、Azureサブスクリプションに**Cognitive Services**リソースを作成してみましょう。

1. 別のブラウザタブで、Azureポータル（https://portal.azure.com）を開き、Microsoftアカウントでサインインします。
3. 「リソースの作成」ボタンをクリックし、「Cognitive Services」を検索し、以下の設定で「Cognitive Services」リソースを作成します。
    - **サブスクリプション**: *ご自身のサブスクリプション*
    - **リソースグループ**: *既存のリソースグループを選択するか、ユニークな名前で新しいリソースグループを作成します。*
    - **リージョン**: *利用可能なリージョンを選択* (例:東日本)
    - **名前**: *ユニークな名前を入力*
    - **価格レベル**: S0
    - **このボックスをオンにすることにより、以下のすべてのご契約条件を読み、同意したものとみなされます**: チェックを入れる。
4. デプロイが完了するのを待ちます。次に、Cognitive Servicesリソースにアクセスし、**概要**ページの左のメニューにある「リソース管理」の下の「キーとエンドポイント」をクリックします。クライアント・アプリケーションからCognitive Servicesリソースに接続するには、エンドポイントとキーが必要になります。

### Cognitive Servicesリソースのキーとエンドポイントの取得

Cognitive Serviceリソースを使用するためには、クライアント・アプリケーションはそのエンドポイントと認証キーを必要とします。

1. Azureポータルで、Cognitive Servicesリソースの**キーとエンドポイント**ページで、リソースの**キー1**をコピーして、**YOUR_COG_KEY**を置き換えて、以下のコードに貼り付けます。
2. リソースの**エンドポイント**をコピーし、**YOUR_COG_ENDPOINT**を置き換えて、以下のコードに貼り付けてください。
4. 以下のコードを実行するには、セルを選択し、セルの左にある**Run cell** (&#9655) ボタンをクリックします。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

キーとエンドポイントの設定が完了したので、コンピュータビジョンサービスを使って画像を解析してみましょう。

次のセルを実行して、*/data/vision/store_cam1.jpg* ファイル内の画像の説明を取得します。

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
from python_code import vision
import os
%matplotlib inline

# Get the path to an image file
image_path = os.path.join('data', 'vision', 'store_cam1.jpg')

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Get a description from the computer vision service
image_stream = open(image_path, "rb")
description = computervision_client.describe_image_in_stream(image_stream)

# Display image and caption (code in helper_scripts/vision.py)
vision.show_image_caption(image_path, description)


なかなかの精度ですね。

別の画像を見てみましょう。

In [ ]:
# Get the path to an image file
image_path = os.path.join('data', 'vision', 'store_cam2.jpg')

# Get a description from the computer vision service
image_stream = open(image_path, "rb")
description = computervision_client.describe_image_in_stream(image_stream)

# Display image and caption (code in helper_scripts/vision.py)
vision.show_image_caption(image_path, description)

これも、提案されたキャプションはかなり正確なようです。

## 画像の特徴を分析する

ここまでは、Computer Visionサービスを使って、数枚の画像に説明的なキャプションを生成してきましたが、それ以外にもできることがあります。Computer Vision サービスには、以下のような詳細情報を抽出する解析機能があります。

- 画像内で検出された一般的なタイプのオブジェクトの位置。
- 画像内の人物の顔の位置とおおよその年齢。
- 画像に「成人向け」、「きわどい」、「グロテスク」なコンテンツが含まれているかどうか。
- 画像の検索を容易にするために、データベースで画像に関連付けられる関連タグ。

以下のコードを実行して、買い物客の画像を解析します。

In [ ]:
# Get the path to an image file
image_path = os.path.join('data', 'vision', 'store_cam1.jpg')

# Specify the features we want to analyze
features = ['Description', 'Tags', 'Adult', 'Objects', 'Faces']

# Get an analysis from the computer vision service
image_stream = open(image_path, "rb")
analysis = computervision_client.analyze_image_in_stream(image_stream, visual_features=features)

# Show the results of analysis (code in helper_scripts/vision.py)
vision.show_image_analysis(image_path, analysis)

## さらに学ぶ

このノートで紹介した機能に加えて、Computer Vision cognitive Serviceには以下の機能があります。

- 画像の中の有名人を識別する。
- 画像内のブランドロゴを検出する。
- 画像内のテキストを読み取るOCR（光学式文字認識）。

Computer Vision cognitive Servicesについてもっと学びたい場合は、[Computer Visionのドキュメント](https://docs.microsoft.com/azure/cognitive-services/computer-vision/)を参照してください。
